In [ ]:
import cv2
import numpy as np
import imageio

# --- Cargar y ajustar la imagen ---
input_path = 'sample_image.jpg'
original = cv2.imread(input_path)

# Calcular dimensiones para presentación
desired_height = 500
aspect = original.shape[0] / original.shape[1]
new_width = int(desired_height / aspect)
new_height = desired_height
frame = cv2.resize(original, (new_width, new_height), interpolation=cv2.INTER_AREA)

# --- Procesamiento de la imagen ---
# 1. Escala de grises
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 2. Desenfoque Gaussiano
blur = cv2.GaussianBlur(frame, (11, 11), 0)

# 3. Realce (sharpen)
sharpen_kernel = np.array([[0, -1,  0],
                           [-1, 5, -1],
                           [0, -1,  0]])
sharpen = cv2.filter2D(frame, -1, sharpen_kernel)

# 4. Bordes Sobel
sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
sobel_x = cv2.convertScaleAbs(sobel_x)
sobel_y = cv2.convertScaleAbs(sobel_y)
sobel = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)

# 5. Bordes Laplaciano
lap = cv2.Laplacian(gray, cv2.CV_64F)
lap = cv2.convertScaleAbs(lap)

# --- Guardar resultados individuales ---
cv2.imwrite('output_gray.png', gray)
cv2.imwrite('output_blur.png', blur)
cv2.imwrite('output_sharpen.png', sharpen)
cv2.imwrite('output_sobel.png', sobel)
cv2.imwrite('output_laplacian.png', lap)

# --- Generación de GIFs de cada “animación” ---
# 1. Transición: original → escala de grises
imageio.mimsave(
    'anim_gray.gif',
    [frame, cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)],
    duration=1.0
)

# 2. Transición: original → desenfoque
imageio.mimsave(
    'anim_blur.gif',
    [frame, blur],
    duration=1.0
)

# 3. Transición: original → realce
imageio.mimsave(
    'anim_sharpen.gif',
    [frame, sharpen],
    duration=1.0
)

# 4. Animación Sobel: Sobel X → Sobel Y → combinado
sobel_frames = [
    cv2.cvtColor(sobel_x, cv2.COLOR_GRAY2BGR),
    cv2.cvtColor(sobel_y, cv2.COLOR_GRAY2BGR),
    cv2.cvtColor(sobel,   cv2.COLOR_GRAY2BGR)
]
imageio.mimsave(
    'anim_sobel.gif',
    sobel_frames,
    duration=0.7
)

# 5. Transición: original → Laplaciano
imageio.mimsave(
    'anim_laplacian.gif',
    [frame, cv2.cvtColor(lap, cv2.COLOR_GRAY2BGR)],
    duration=1.0
)
